# Objective

This project demonstrates a Proof of Concept (POC) for automated invoice analysis using a multi-LLM approach built on OpenAI's GPT-4o and GPT-4o-mini models. The system is designed to handle invoices with diverse formats and layouts from multiple vendors. It leverages GPT-4o’s multimodal capabilities to analyze invoice images and identify issues such as missing vendor or bank details, tax information, image clarity problems, and unusual quantities or amounts. These findings are returned as structured error messages and observations.

The error messages are then used to query a policy vector store, retrieving two relevant policy chunks. GPT-4o-mini subsequently analyzes the errors in the context of these policy clauses to determine if any violations have occurred. The response is generated in a consistent JSON format and includes a validation_summary, risk assessment (with references to relevant policy clauses), and suggestive_action derived from the policies.

This project draws inspiration from the Model Context Protocol (MCP) concept to enforce clean and predictable interactions between LLMs. By using OpenAI's function calling feature, a strict schema structure is applied to both model outputs and inputs, ensuring reliable handoff and alignment across model stages. This structure not only enhances consistency but also makes the multi-step reasoning process more transparent and robust.



# Installations

In [ ]:
!pip install openai langchain
!pip install -U langchain langchain-community
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.71
    Uninstalling langchain-core-0.3.71:
      Successfully uninstalled langchain-core-0.3.71
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.26
    Uninstalling langchain-0.3.26:
      Successfully uninstalled langchain-0.3.26
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [ ]:
import os
from google.colab import userdata
import base64
import openai
import json
import re
from langchain_community.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Global Initializations

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI()

In [ ]:
!unzip policy_store.zip -d policy_store

Archive:  policy_store.zip
   creating: policy_store/8f819a0e-86ad-4122-92b7-8bf3a2c15fdd/
  inflating: policy_store/chroma.sqlite3  
  inflating: policy_store/8f819a0e-86ad-4122-92b7-8bf3a2c15fdd/data_level0.bin  
  inflating: policy_store/8f819a0e-86ad-4122-92b7-8bf3a2c15fdd/link_lists.bin  
  inflating: policy_store/8f819a0e-86ad-4122-92b7-8bf3a2c15fdd/header.bin  
  inflating: policy_store/8f819a0e-86ad-4122-92b7-8bf3a2c15fdd/length.bin  


In [ ]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store=Chroma(
        persist_directory="policy_store",
        embedding_function=embedding_model,
        collection_name="policy_collection"
    )

retriever = vector_store.as_retriever(search_kwargs={"k": 2})

/tmp/ipython-input-6-1218783993.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-6-1218783993.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store=Chroma(


In [ ]:
schema={
  "name": "validate_invoice",
  "description": "Returns invoice validation results",
  "parameters": {
    "type": "object",
    "properties": {
      "header": {
        "type": "object",
        "properties": {
          "invoice_no": {"type": "string"}
        },
        "required": ["invoice_no"]
      },
      "validations": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "key_name": {"type": "string"},
            "compliant": {"type": "boolean"},
            "observation": {"type": "string"},
            "error": {"type": "string"}
          },
          "required": ["key_name", "compliant", "observation", "error"]
        }
      }
    },
    "required": ["header", "validations"]
  }
}

In [ ]:
report_schema = {
    "name": "generate_risk_report",
    "description": "Generates a risk report for non-compliant invoice validations including observations, inferred risks, and suggestions.",
    "parameters": {
        "type": "object",
        "properties": {
            "reports": {
                "type": "array",
                "description": "List of analyzed results.",
                "items": {
                    "type": "object",
                    "properties": {
                        "key_name": {
                            "type": "string",
                            "description": "The field of the invoice that failed compliance."
                        },
                        "Validation_Summary": {
                            "type": "string",
                            "description": "Detailed observation from the check."
                        },
                        "Risk": {
                            "type": "string",
                            "description": "Policy clause(s) violated and justification(s), or 'No policy violated.'"
                        },
                        "Suggestive_Action": {
                            "type": "string",
                            "description": "General corrective action based on the violated clause(s), or 'No action needed'"
                        }
                    },
                    "required": ["Validation_Summary", "Risk", "Suggestive_Action"]
                }
            }
        },
        "required": ["reports"]
    }
}



# Invoice Processor

In [ ]:
def encode_image_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def process_invoice_image(image_path):
  prompt="""
  # CONTEXT

You are a **highly expert compliance validator**, specialized in extracting, verifying, and validating critical fields from **invoice images** with maximum precision and accuracy. Your role is to analyze diverse invoice formats and strictly evaluate compliance based on the defined criteria.

You MUST extract data only from the image, make NO assumptions, and respond with **ONLY** the output JSON in the exact schema provided.

---

# CONDITIONS FOR COMPLIANCE

1. **Clear Image**
   - Image MUST NOT be blurry.
   - Image MUST NOT be cut off more than 50%.

2. **Vendor Identification**
   - Vendor Name MUST be present.
   - Vendor Address MUST be present.
   - A **corporate** email (not personal like Gmail/Yahoo/etc.) MUST be present.
   - Beneficiary Name and Bank Account Details MUST be present.
   - Beneficiary Name MUST EXACTLY MATCH Vendor Name **(ignore case, punctuation, and whitespace only)**.

3. **Tax Information**
   - A tax **percentage** or **amount** MUST be present.
   - A **Tax ID** MUST be present, starting with `"TIN"` followed by **exactly 3 digits** (e.g., `"TIN237"`).

4. **Purchase Details**
   - There MUST be a **quantity column**, with recognizable headers like `Qty`, `qty`, or `Quantity`.
   - ALL quantity values **MUST be less than 10000**.
   - The **Total Due** amount MUST be **less than $1,000,000**.

5. **Invoice Number**
   - Must start with `"INV"` (case-sensitive).
   - Extract the full Invoice Number starting with `"INV"`.

---

# INSTRUCTIONS

1. **Step-by-step evaluation for each key:**
   - First, analyze the field thoroughly.
   - Fill the `observation` key with a clear, concise **full factual summary** of what was observed.
   - Based on the observation, validate against the compliance conditions.
   - If any rule fails, set `compliant: false` and explain the exact issue in the `error` key.
   - If all conditions pass, set `compliant: true`, set `error` to an empty string, and still provide a useful `observation`.

2. **Cross-verification logic:**
   - After assigning `compliant: false`, double-check your observation and ensure the error truly reflects a failed condition.
   - After assigning `compliant: true`, **again cross-verify** if all conditions have genuinely passed as per the observation.
   - If a mismatch is found in your own reasoning, **correct it**, update the keys, and only then proceed to the next field.
   - Use this strict Chain-of-Thought validation for every field.

3. **Vendor and Beneficiary name comparison:**
   - Perform an **exact match**, ignoring case, whitespace, and punctuation.
   - DO NOT allow approximate or fuzzy matches.
   - Provide the actual names in the error message like:
     `"Vendor name and Beneficiary name did not match. Vendor: 'Acme Corp', Beneficiary: 'Acmexx Ltd Corp'"`

# INSTRUCTIONS – STRICT LOGIC MUST BE FOLLOWED, NO EXCEPTIONS

For each key, follow the steps **in exact order**:

1. **Analyze the image carefully** and extract information only if clearly visible and labeled.
2. **Populate the `observation`** field with a factual summary of what you observed (even if it's non-compliant).
3. **Check the conditions for compliance strictly** (see section "# CONDITIONS FOR COMPLIANCE").
4. If **any condition fails**, set `"compliant": false` and write the exact reason in the `"error"` field.
5. If **all conditions pass**, set `"compliant": true`, set `"error": ""`, but still fill the `observation`.
6. **Cross-verify your decision**:
   - If you set `"compliant": false`, make sure the `observation` clearly shows a rule is broken. Also provide clear explanations in the `error` field.
   - If you set `"compliant": true`, make sure the `observation` clearly holds all the values and satisifies all the conditions.
   - If `"compliant": true`, re-check in the `observation` field that no condition was missed and no value was missed.
   - If you find any value missing or not found in the observation field, set `"compliant": false` and write a clear explanation in the `error` field.
   - If a mismatch is found in your reasoning, **correct the observation, error, and compliance** before proceeding.
   - Use this strict Chain-of-Thought validation for every field.
   - ALERT: Avoid common mistakes like: even after marking a value as 'not found' in `observation` field, no error was provided and `compliant: true` was set.
   - You will be heavily penalized by law if you fail to comply.

---

## FIXED FORMAT FOR observation FIELD (Mandatory for All Keys)

1. For every validation key, you must populate the observation field with a **structured, factual summary listing each required sub-element explicitly**.
2. If any field is missing, write "not found" in the observation.
3. If a value is invalid (e.g., wrong email format or Tax ID), include it and indicate the issue.
4. Never leave observation vague or generic. Always be explicit and detailed.
5. Do not assume or invent values. Only write what is clearly present on the invoice image.

### Format Examples:

#### Vendor Identification

Vendor Name: 'Acme Corp'
Beneficiary Name: 'Acme Corp'
Vendor Address: '123 Main Street, NY'
Vendor Email: 'billing@acme.com' (corporate)
Bank Account Details: Present

If some fields are missing or invalid:

Vendor Name: 'Acme Corp'
Beneficiary Name: 'Acme Corp'
Vendor Address: not found
Vendor Email: 'acme@gmail.com' (personal)
Bank Account Details: Present


#### Tax Information

Tax ID: 'TIN100'
Tax Amount: $540
Tax Percentage: 18%

If some fields are missing:

Tax ID: not found
Tax Amount: not found
Tax Percentage: 18%

#### Purchase Details

Quantity Column: Found
Quantity Values: [5, 20, 9999]
Total Due: $850,000


---

## SPECIFIC KEY-WISE INSTRUCTIONS

### Clear Image
- DO NOT assume image is clear unless visibly verified.
- Check for:
  - Obvious blur or distortion.
  - Whether more than 50% of the image is cropped/cut off.
- If either condition is true, set `"compliant": false` with an error like:
  `"Image is too blurry"` or `"More than 50% of the image is cut off"`.

---

### Vendor Identification

- Identify the **Vendor(Seller) — the party issuing the invoice**, NOT the buyer.
  - DO NOT extract from sections labeled: `Billed To`, `Client`, or `Customer`.
  - ONLY extract from sections labeled: `From`, `Vendor`, `Seller`, `Issued By`, or clearly positioned at the top of the invoice as sender.
- You **MUST Extract and verify the following 6 conditions**. If any one is missing or invalid, mark `"compliant": false`.
  1. Vendor **name** (from correct label/location only)
  2. Vendor **address** (must be present)
  3. A **corporate email id** (e.g., accounts@company.com; not Gmail, Yahoo, Rediff, etc.)
  4. **Beneficiary name**
  5. **Bank account details** (bank name, account number)
  6. Vendor Name must match Beneficiary Name exactly. Follow the below rules for comparison.

  #### Vendor vs Beneficiary Name Matching Logic

- Compare **Vendor Name vs Beneficiary Name** using following rules:
  - Must match **exactly after ignoring**:
    - Case (e.g., `ABC Ltd` and `abc ltd` are considered equal)
    - Leading/trailing spaces
    - Punctuation (e.g., `.`, `,`. `ABC Inc.` and `ABC Inc` are considered equal)
  - DO NOT ALLOW:
    - Extra or Missing letters (e.g., `ABC Ltd` not equal to `ABCxx Pvt Ltd`)
    - Matching with Client, Buyer, or Customer name

**All of the above conditions must be satisfied to mark this field as compliant.**
If even one of these conditions fails, set "compliant": false and write a clear explanation in the error field.
If compliant:
- Mention both names in `observation`.
- `error = ""`

If mismatch:
- Show both names in `error`, e.g.:
  `"Vendor name and Beneficiary name do not match. Vendor: 'ABC Inc', Beneficiary: 'ABCxx Ltd Inc'"`

---

### Tax Information

- You **MUST Extract and verify the following 2 conditions**. If any one is missing or invalid, mark `"compliant": false`.
1. You **MUST identify a Tax ID** with the format `TIN` followed by **exactly 3 digits**.
  - Example: `"TIN101"` is valid. `"TIN12"` or `"TIN1234"` is invalid.
2. You MUST find either:
  - A Tax **percentage** (e.g., "18%"), OR
  - A Tax **amount** (e.g., "$500")
- DO NOT assume presence of Tax ID. If missing, set `compliant: false`.

Common mistakes to avoid:
- Saying Tax ID is present when it’s not present
- Setting Tax ID as 'not found' in the `observation` but still setting compliant: true` and leaving error message as blank.
- Accepting incorrect Tax ID formats
- Making up tax amounts from totals

---

### Purchase Details

- Look for a column labeled:
  - `Qty`, `qty`, `Quantity`, or similar
- Extract **ALL quantity values**.
  - Ensure **all values are numeric and less than 10000**
  - Check carefully. **Do not hallucinate** fake large numbers.
- Check that **Total Due** (grand total or net amount) is **less than $1,000,000**
- DO NOT extract from irrelevant tables

If any quantity is greater than 10000 or total is greater than $1,000,000, mark non-compliant with a clear error:
- `"Quantity 25000 exceeds limit of 10000"`
- `"Total due $2,300,000 exceeds maximum allowed"`

---

### Invoice Number

- Look for a label like `"Invoice #"`, `"Invoice No"`, etc.
- The invoice number **must begin with `"INV"`**.
  - Extract entire string starting with `"INV"` (e.g., `"INV-10023-A"`)

---

5. **Response Format**
   - Output only the JSON below, no explanations.
   - DO NOT modify key names, casing, or schema format.

---

# OUTPUT SCHEMA

```json
{
  "header": {
    "invoice_no": "string"
  },
  "validations": [
    {
      "key_name": "Clear Image",
      "compliant": true/false,
      "observation": "string",
      "error": "string"
    },
    {
      "key_name": "Vendor Identification",
      "compliant": true/false,
      "observation": "string",
      "error": "string"
    },
    {
      "key_name": "Tax Information",
      "compliant": true/false,
      "observation": "string",
      "error": "string"
    },
    {
      "key_name": "Purchase Details",
      "compliant": true/false,
      "observation": "string",
      "error": "string"
    }
  ]
}

This is a strict validation task. No assumptions. No hallucinations. If a field is missing or does not meet the criteria exactly, mark it non-compliant with clear justification. All validations must be auditable, transparent, and logically consistent.
  """
  base64_img = encode_image_to_base64(image_path)

  system_message="""
You are a **highly expert** compliance validator specialized in analyzing invoice images with absolute precision. Your goal is to detect any discrepancies, compliance violations, or anomalies in the invoice. Use keen observation and sound judgment to assess the document strictly according to standard invoicing and regulatory norms.

You MUST STRICTLY FOLLOW every instruction in the user message WITHOUT deviation.
- Do NOT guess or assume any missing information.
- You MUST ONLY mark a field as TRUE if the criteria are 100% satisfied.
- You MUST provide explicit error messages for any field marked FALSE.
- Your responses MUST strictly follow the specified JSON function schema and format.
- You MUST ALWAYS respond by calling the designated function with perfectly structured JSON arguments.
- Do NOT output free text or explanations outside the function call.
- Your output must be deterministic, consistent, and unambiguous.
- No extra commentary, no apologies, no filler text.
- Follow the user’s instructions with absolute accuracy every single time.

You will be heavily penalized by law if you fail to comply. Your output is used for critical compliance decisions.
"""


  messages = [
      {
          "role": "system",
          "content": system_message
      },
      {
          "role": "user",
          "content": [
              {"type": "text", "text": prompt},
              {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"}}
          ]
      }
  ]

  response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0.2,
        functions=[schema],
        function_call={"name": "validate_invoice"}
    )
  # return response.choices[0].message.function_call
  function_call=response.choices[0].message.function_call
  print(function_call)
  if function_call:
      arguments = function_call.arguments
      parsed_output = json.loads(arguments)
      return parsed_output
  else:
      # fallback: raw content if no function call (unlikely if forced)
      # raw = response.choices[0].message.content.strip()
      return {}

# Policy Analyzer

In [ ]:
def policy_reasoner(input_json):
  system_message="""
  You are an expert compliance risk analyst.

You will receive a list of dictionaries in JSON format. The dictionaries represent specific validation failures from an invoice compliance checker. Each failure dictionary contains:

1. key_name : A string value indicating the field in the invoice which failed the compliance. (e.g 'Vendor Identification', 'Tax Information', 'Puchase Details' etc.)
2. compliant : A boolean value showing if compliant or not.
3. observation : A full factual summary of the observations.
4. error : An error message that explains the issue.
5. A 'policy_chunks' field containing two policy clause chunks. These may or may not be violated by the observation. It is a dictionary that contains:
   - content : The text extract from the policy document along with the Clause number.
   - metadata : A dictionary that contains Page_No (page number of document where teh content is present) , Chunk_No (chunk number) and Clause_No (clause number of the policy).

Your task is to:
1. Extract and analyze the error message mentioned in the error key.
2. Analyze the **two provided policy_chunks** and determine which clause(s), if any, are violated.
   - The error message may violate one, both, or neither.
   - Only consider a policy violated if its content **clearly applies** to the error message.
3. For each violated clause, generate a **brief one-line justification** and **cite the clause number** (e.g. 'Clause 4').
4. Suggest a **general corrective action** based on the violated clause(s).

Important constraints:
- Use **only** the two provided policy chunks. Do **not** make assumptions or hallucinate.
- If no policy is violated, state "No policy violated" in the risk field and "No action needed" in the suggestive_action field.
- Be strict: only mark a violation if the language clearly applies to the observation.

Output a JSON array, one object per failed validation. Each object must have the following 4 keys:

- 'key_name': the field in the invoice which failed the compliance.
- 'Validation_Summary': a short, human-readable statement summarizing both the specific validation failure (error key) and its observed context (observation key).
- 'Risk': a single-line justification referencing each violated policy clause, including the clause number and reason for non-compliance. If no clause is violated, return 'No policy violated.'
- 'Suggestive_Action': a general corrective step based on the policy language. Or 'No action needed.' if no policy is violated.

Only use the given input structure and policy content.
  """
  user_message=f"""
  # CONTEXT
You are an expert compliance risk analyst.

You will receive a list of dictionaries in JSON format. The dictionaries represent specific validation failures from an invoice compliance checker. Each failure dictionary contains:

1. key_name : A string value indicating the field in the invoice which failed the compliance. (e.g 'Vendor Identification', 'Tax Information', 'Puchase Details' etc.)
2. compliant : A boolean value showing if compliant or not.
3. observation : A full factual summary of the observations.
4. error : An error message that explains the issue.
5. A 'policy_chunks' field containing two policy clause chunks. These may or may not be violated by the observation. It is a dictionary that contains:
   - content : The text extract from the policy document along with the Clause number.
   - metadata : A dictionary that contains Page_No (page number of document where teh content is present) , Chunk_No (chunk number) and Clause_No (clause number of the policy).

Follow the given instrcutions carefully to analyze the dictionaries and generate a response in JSON format.

# INSTRUCTIONS

Your task is to do the following for **each failure dictionary**:

1. Extract and analyze the error message mentioned in the error key.
2. Analyze the **two provided policy_chunks** and determine which clause(s), if any, are violated.
   - The error message may violate one, both, or neither.
   - Only consider a policy violated if its content **clearly applies** to the error message.
3. For each violated clause, generate a **brief one-line justification** and **cite the clause number** (e.g. 'Clause 4').
4. Suggest a **general corrective action** based on the violated clause(s).


Do not guess, assume, or use knowledge outside the given `policy_chunks`.

Output a JSON array, one object per failed validation. Each object must have the following 4 keys:

- 'key_name': the field in the invoice which failed the compliance.
- 'Validation_Summary': a short, human-readable statement summarizing both the specific validation failure (error key) and its observed context (observation key).
- 'Risk': a single-line justification referencing each violated policy clause, including the clause number and reason for non-compliance. If no clause is violated, return 'No policy violated.'
- 'Suggestive_Action': a general corrective step based on the policy language. Or 'No action needed.' if no policy is violated.

Only use the given input structure and policy content.

---

# INPUT

Here is the input:

{input_json}


---

# OUTPUT SCHEMA

```json
[
   {{
      \"key_name\": \"string\",
    \"validation_summary\": \"string\",
    \"risk\": \"string\",
    \"suggestive_action\": \"string\"

   }}
   // ... might have more dictionaries with the same structure


]
"""
  system_message = {
    "role": "system",
    "content": system_message
    }
  prompt={
      "role": "user",
      "content": user_message
      }
  messages=[system_message,prompt]

  response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2,
        functions=[report_schema],
        function_call={"name": "generate_risk_report"}
    )
  # return response.choices[0].message.function_call
  function_call=response.choices[0].message.function_call
  print(function_call)
  if function_call:
      arguments = function_call.arguments
      parsed_output = json.loads(arguments)
      return parsed_output
  else:
      # fallback: raw content if no function call (unlikely if forced)
      # raw = response.choices[0].message.content.strip()
      return {}


# Formatter

In [ ]:
def format_risk_report(data):
    sections = []
    report_items = data.get('reports', [])

    for item in report_items:
        section = (
            f"Validation Summary: {item.get('Validation_Summary', 'N/A')}\n"
            f"Risk: {item.get('Risk', 'N/A')}\n"
            f"Suggestive Action: {item.get('Suggestive_Action', 'N/A')}"
        )
        sections.append(section)

    return "\n\n" + ("\n" + "=" * 50 + "\n\n").join(sections)

# Retrieval

In [ ]:
import json

def policy_retriever(validations, retriever):
    enriched = []

    for item in validations:
        if not item.get("compliant") and item.get("error", "").strip():
            query = item["error"]
            relevant_docs = retriever.get_relevant_documents(query)
            policy_chunks = []

            for doc in relevant_docs:
                policy_chunks.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })

            item["policy_chunks"] = policy_chunks
            enriched.append(item)

    return json.dumps(enriched, indent=2)


# Pipeline

In [ ]:
def run_pipeline(invoice_path):
  print(f"Processing: {invoice_path}")
  inv_result = process_invoice_image(invoice_path)
  print("GPT-4o Output:", inv_result)

  if inv_result!={}:
    invoice_num=inv_result["header"]["invoice_no"]
    validations=inv_result["validations"]
    is_compliant = all(v["compliant"] for v in validations)
    if is_compliant:
      answer=invoice_num +":\n\n The invoice is compliant.\n\n"
      return answer
    else:
      print("Non-compliant. Running policy check...")
      retrieved_res=policy_retriever(validations, retriever)
      print("Analyzing...")
      analysis = policy_reasoner(retrieved_res)
      print("Compliance Analysis:", analysis)
      print("Parsed analysis:",analysis)
      outcome=format_risk_report(analysis)
      answer=invoice_num +":"+ outcome
      return answer




# Main

## Invoice 1

In [ ]:
invoice="inv-1.jpg"
answer=run_pipeline(invoice)

Processing: inv-1.jpg
FunctionCall(arguments='{"header":{"invoice_no":"INV-20250722-1043"},"validations":[{"key_name":"Clear Image","compliant":true,"observation":"Image is clear and not cut off.","error":""},{"key_name":"Vendor Identification","compliant":true,"observation":"Vendor Name: \'TechNova Solutions Inc.\' Beneficiary Name: \'TechNova Solutions Inc.\' Vendor Address: \'1234 Digital Way, Suite 400, San Francisco, CA 94107\' Vendor Email: \'billing@technova-solutions.com\' (corporate) Bank Account Details: Present","error":""},{"key_name":"Tax Information","compliant":false,"observation":"Tax ID: not found Tax Amount: $220.36 Tax Percentage: 8.5%","error":"Tax ID is missing."},{"key_name":"Purchase Details","compliant":true,"observation":"Quantity Column: Found Quantity Values: [25, 15, 5, 10, 1, 3, 25, 30] Total Due: $2,812.86","error":""}]}', name='validate_invoice')
GPT-4o Output: {'header': {'invoice_no': 'INV-20250722-1043'}, 'validations': [{'key_name': 'Clear Image', 'co

### Report

In [ ]:
print(answer)

INV-20250722-1043:

Validation Summary: Tax ID is missing, as indicated by the observation that the Tax ID was not found, along with the tax amount and percentage provided.
Risk: Clause 5: Invoices must include a valid tax identification number where required, and the absence of the Tax ID constitutes non-compliance.
Suggestive Action: Ensure that all invoices include a valid tax identification number as per jurisdictional requirements.


## Invoice 2

In [ ]:
invoice="inv-2.jpg"
answer=run_pipeline(invoice)

Processing: inv-2.jpg
FunctionCall(arguments='{"header":{"invoice_no":"INV-20250726-3321"},"validations":[{"key_name":"Clear Image","compliant":true,"observation":"Image is clear and not cut off.","error":""},{"key_name":"Vendor Identification","compliant":false,"observation":"Vendor Name: \'ByteCore Systems Ltd.\' Beneficiary Name: \'ByteCorexx Systems Pvt Ltd.\' Vendor Address: \'78 Silicon Drive, Tech Park, Austin, TX 73301\' Vendor Email: \'billing@bytecore.com\' (corporate) Bank Account Details: Present","error":"Vendor name and Beneficiary name do not match. Vendor: \'ByteCore Systems Ltd.\', Beneficiary: \'ByteCorexx Systems Pvt Ltd.\'"},{"key_name":"Tax Information","compliant":true,"observation":"Tax ID: \'TIN123\' Tax Amount: $811.75 Tax Percentage: 8.5%","error":""},{"key_name":"Purchase Details","compliant":false,"observation":"Quantity Column: Found Quantity Values: [5, 100000, 7, 10, 3] Total Due: $11,508,400","error":"Quantity 100000 exceeds limit of 10000. Total due $11

### Report

In [ ]:
print(answer)

INV-20250726-3321:

Validation Summary: Vendor name and Beneficiary name do not match. Vendor: 'ByteCore Systems Ltd.', Beneficiary: 'ByteCorexx Systems Pvt Ltd.'
Risk: Clause 4: Discrepancy between vendor name and beneficiary details must be resolved prior to disbursal.
Suggestive Action: Ensure that the vendor name on the invoice matches the beneficiary name in payment instructions.

Validation Summary: Quantity 100000 exceeds limit of 10000. Total due $11,508,400 exceeds maximum allowed.
Risk: Clause 6: Transactions involving unusually large quantities may be subject to enhanced review.
Suggestive Action: Review the invoice for unusually large quantities and provide supplementary documentation or business justification for approval.


## Invoice 3

In [ ]:
invoice="inv-3.jpg"
answer=run_pipeline(invoice)

Processing: inv-3.jpg
FunctionCall(arguments='{"header":{"invoice_no":"INV-20250726-7789"},"validations":[{"key_name":"Clear Image","compliant":true,"observation":"Image is clear and not cut off.","error":""},{"key_name":"Vendor Identification","compliant":true,"observation":"Vendor Name: \'OfficeEdge Supplies Co.\' Beneficiary Name: \'OfficeEdge Supplies Co.\' Vendor Address: \'98 Stationery Lane, Midtown, Chicago, IL 60601\' Vendor Email: \'accounts@officeedge.com\' (corporate) Bank Account Details: Present","error":""},{"key_name":"Tax Information","compliant":true,"observation":"Tax ID: \'TIN567\' Tax Amount: $358.70 Tax Percentage: 8.5%","error":""},{"key_name":"Purchase Details","compliant":true,"observation":"Quantity Column: Found Quantity Values: [2, 3, 10, 4, 1] Total Due: $4,578.70","error":""}]}', name='validate_invoice')
GPT-4o Output: {'header': {'invoice_no': 'INV-20250726-7789'}, 'validations': [{'key_name': 'Clear Image', 'compliant': True, 'observation': 'Image is cle

### Report

In [ ]:
print(answer)

INV-20250726-7789:

 The invoice is compliant.




# Conclusion

This proof of concept (POC) demonstrates a multi-LLM pipeline for automated invoice analysis using GPT-4o and GPT-4o-mini. Despite wide variability in invoice formats, key data points were extracted, issues identified, and evaluations performed against organizational policies through structured outputs and enforced schema.

The architecture is inspired by the Model Context Protocol (MCP), emphasizing consistent interfaces and reliable structured data exchange between sequential model stages.

While GPT-4o is effective in multimodal tasks, it is not optimized for OCR. Consequently, some inaccuracies were observed during text extraction from invoice images. Due to time and budget constraints, specialized OCR tools such as Google Document AI or Amazon Textract were not integrated. Prompt refinement and reruns were used to mitigate these limitations. For production environments, the integration of dedicated OCR solutions is strongly recommended.

This POC was intentionally designed as a lightweight, self-contained system to enable rapid prototyping of multi-LLM orchestration, schema enforcement, and policy-aware validation. Thoughtful trade-offs were made to accelerate development while laying a solid foundation for more robust, production-ready implementations.